## checklist

### 진행예정
 1) 추가필요 열: 플랫폼명, 날짜 \
 2) 타 플랫폼과 합쳤을 때 형식 통일하도록 data type과 Nan 처리방법 등 정의 \
 3) 실행소요시간 마지막에 매직커맨드로 체크 \
 4) import는 설치 및 실행하는 거 settings.py 등으로 세팅하기 \
 5) MySQL로 나눠서 관리할 거면, 이름, 주소, 리뷰수 등은 순환주기 하루 어떤가? \
    리뷰수가 request를 한 번 더 보내야 해서 시간을 잡아먹는다 \
    그러면 어차피 스파이더를 두 개로 쪼개야하기는 하겠다
 6) 쌓다가 원하는 호텔 원하는 가격 등 조건 방 나오면 알림 전달 서비스 추가도 좋을듯!


### 완료
 1) page바꿔가면서 했을 때 상품들 연속성 CHK (limit=50에 따른 상품 누락 여부 확인) -> 이상없음. \
 2) page=1, 2 이상 구분할 때 [4], [1]이 아니라 '전체'를 가져오게 해야겠다. 시간 저녁되니까 숫자로 안된다. -> 해결 \
 3) 리뷰수 추가 완료

### 추후 체크 디버깅 예상
 1) 예약만실 시 그대로 반영 or 삭제

In [1]:
import numpy as np
import pandas as pd

In [1]:
# 1. 프로젝트 생성

In [2]:
!scrapy startproject goodchoice

New Scrapy project 'goodchoice', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/03_ml/goodchoice

You can start your first spider with:
    cd goodchoice
    scrapy genspider example example.com


In [3]:
# 2. xpath 찾기

In [1]:
# 1) gu별로 페이지 받았을 때, 호텔리스트 받기 위한 xpath

In [ ]:
links = response.xpath('//*[@id="poduct_list_area"]/li/a/@href').extract()

In [ ]:
# 2) 호텔별로 상세페이지 링크 받았을 때, 각호텔별 상세정보 받기 위한 xpath

In [6]:
# xpath 찾기 전에 필요한 사전 import & settings
import scrapy, requests
from scrapy.http import TextResponse
import datetime
import re


KST = datetime.timezone(datetime.timedelta(hours=9))
today = datetime.datetime.now(tz=KST)
start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
            datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
persons = 2
gu_code = ['2012']


# 2012 하나에 대해 링크들 가져오기
for gu in gu_code:
        url = f'https://www.goodchoice.kr/product/search/2/{gu}?sort=HIT\
&sel_date={start_date}&sel_date2={end_date}&persons={persons}'
        req = requests.get(url)
        response = TextResponse(req.url, body=req.text, encoding='utf-8')
        links = response.xpath('//*[@id="poduct_list_area"]/li/a/@href').extract()

In [22]:
# 2012 링크들 중 3개에 대해서 테스트 진행 -> Success
for link in links[:1]:
    req = requests.get(link)
    response = TextResponse(req.url, body=req.text, encoding='utf-8')
    
    # name
    name = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h2/text()')[0].extract()
    # level
    level = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/span/text()')[0].extract()
    # score
    try:
        score = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/div[1]/span/text()')[0].extract()
    except:
        score = ['점수없음']
    # location    
    location = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/p[2]/text()')[0].extract()
    # roomtype
    try:
        roomtype = response.xpath('//*[@id="product_filter_form"]/article/div/strong/text()')[1:].extract()
    except:
        roomtype = ['예약만실']

    # price
    price = []
    for num in range(2, 2+len(roomtype)):
        try:
            price.append(response.xpath(f'//*[@id="product_filter_form"]/article/div[{num}]/div[3]/div/div/div/p[2]/b/text()')[0].extract())
        except:
            price.append('다른날짜확인')

    # review (link에 직접 담겨있지 않으며 link url의 hotel_id로 request url 재구성 필요)
    hotel_id = re.findall("ano\=([0-9]+)\&", response.url)[0]
    review_url = 'https://www.goodchoice.kr/product/get_review_non'
    params = {"page": 0, "ano": hotel_id}
    response2 = requests.post(review_url, params)
    review = re.findall('"count":([0-9]+)', response2.text)[0]

    # link
    hotel_desc_url = response.url
    
    
    print(name), print(level), print(score), print(location),print(roomtype), print(price), print(review), print(link) 

[공동구매] 인터컨티넨탈 서울 코엑스
5성급
9.6
서울 강남구 삼성동 159
['[주중! 공동구매 머선129 패키지] 수피리어 더블', '[주중! 공동구매 머선129 패키지] 수피리어 트윈', '[주중! 공동구매 머선129 패키지-2인 조식] ...', '[주중! 공동구매 머선129 패키지-2인 조식] ...', '[미니바 습격 패키지] 수피리어 더블', '[미니바 습격 패키지] 수피리어 트윈', '[봄캉스 특가! Fresh한 시작] 수피리어 더블', '[봄캉스 특가! Fresh한 시작] 수피리어 트윈', '수피리어 더블', '수피리어 트윈', '[Stay and Eat more 패키지] 수피리어 더블', '[Stay and Eat more 패키지] 수피리어 트윈', '[술기로운 생활 패키지] 수피리어 더블', '[술기로운 생활 패키지] 수피리어 트윈', '[비타 스프링 패키지] 수피리어 더블', '[비타 스프링 패키지] 수피리어 트윈', '[Lovely Bubbly 키즈 패키지] 수피리어 더블', '[Lovely Bubbly 키즈 패키지] 수피리어 트윈', '[반짝이는 연애일기 패키지] 수피리어 더블', '[반짝이는 연애일기 패키지] 수피리어 트윈']
['119,000원', '119,000원', '159,000원', '159,000원', '185,000원', '185,000원', '202,000원', '202,000원', '211,750원', '211,750원', '212,960원', '212,960원', '220,000원', '220,000원', '230,000원', '230,000원', '250,000원', '250,000원', '260,000원', '260,000원']
261
https://www.goodchoice.kr/product/detail?ano=6980&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12


In [18]:
# 3. items.py

In [3]:
%%writefile goodchoice/goodchoice/items.py
import scrapy


class GoodchoiceItem(scrapy.Item):
    platform = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review = scrapy.Field()
    location = scrapy.Field()
    roomtype = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting goodchoice/goodchoice/items.py


In [21]:
# 4. spider.py

In [8]:
%%writefile goodchoice/goodchoice/spiders/spider.py
import scrapy, requests
from scrapy.http import TextResponse
import datetime
import re
from goodchoice.items import GoodchoiceItem


class GoodchoiceSpider(scrapy.Spider):
    name = "Goodchoice"
    allow_domain= ["goodchoice.kr/"]
    KST = datetime.timezone(datetime.timedelta(hours=9))
    today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
    start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
    end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
    staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
                datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
    persons, staydays = 2, 1
    start_urls= ["https://www.dailyhotel.com/"]

    
    def start_requests(self):
        KST = datetime.timezone(datetime.timedelta(hours=9))
        today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
        start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
        end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
        persons = 2
        gu_code = ['2012', '2019', '2016', '2014', '2015', '2020', '2018', '2017', '2021']
        for gu in gu_code:
            start_url = f'https://www.goodchoice.kr/product/search/2/{gu}?sort=HIT\
            &sel_date={start_date}&sel_date2={end_date}&persons={persons}'
            yield scrapy.Request(start_url, callback=self.parse)
            
            
    def parse(self, response):
        links = response.xpath('//*[@id="poduct_list_area"]/li/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
            
    def parse_content(self, response):
        # name
        name = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h2/text()')[0].extract()
        # level
        level = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/span/text()')[0].extract()
        # score
        try:
            score = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/div[1]/span/text()')[0].extract()
        except:
            score = ['점수없음']
        # location    
        location = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/p[2]/text()')[0].extract()
        # roomtype
        try:
            roomtype = response.xpath('//*[@id="product_filter_form"]/article/div/strong/text()')[1:].extract()
        except:
            roomtype = ['예약만실']
        # price
        price = []
        for num in range(2, 2+len(roomtype)):
            try:
                price.append(response.xpath(f'//*[@id="product_filter_form"]/article/div[{num}]/div[3]/div/div/div/p[2]/b/text()')[0].extract())
            except:
                price.append('다른날짜확인')
        # review (link에 직접 담겨있지 않으며 link url의 hotel_id로 request url 재구성 필요)
        hotel_id = re.findall("ano\=([0-9]+)\&", response.url)[0]
        review_url = 'https://www.goodchoice.kr/product/get_review_non'
        params = {"page": 0, "ano": hotel_id}
        response2 = requests.post(review_url, params)
        review = re.findall('"count":([0-9]+)', response2.text)[0]
        # link
        hotel_desc_url = response.url
        # platform
        platform = "여기어때"
        
        
        item = GoodchoiceItem()
        for i in range(len(roomtype)):
            item['platform'] = platform
            item['name'] = name
            item['level'] = level
            item['score'] = score
            item['review'] = review
            item['location'] = location
            item['roomtype'] = roomtype[i]
            item['price'] = price[i]
            item['link'] = hotel_desc_url
            yield item

Overwriting goodchoice/goodchoice/spiders/spider.py


In [ ]:
# 5. settings.py: robots.txt 여부 파악

In [24]:
# 6. 프로젝트 실행

In [5]:
%pwd

'/home/ubuntu/python3/notebook/03_ml'

In [9]:
%%writefile run.sh
cd goodchoice
rm goodchoice.csv
scrapy crawl Goodchoice -o goodchoice.csv 

Overwriting run.sh


In [10]:
!source run.sh

2021-03-11 18:08:56 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: goodchoice)
2021-03-11 18:08:56 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jan 15 2021, 05:57:06) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-11 18:08:56 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-11 18:08:56 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'goodchoice',
 'NEWSPIDER_MODULE': 'goodchoice.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['goodchoice.spiders']}
2021-03-11 18:08:56 [scrapy.extensions.telnet] INFO: Telnet Password: 5269977bbaea0b78
2021-03-11 18:08:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsa

2021-03-11 18:08:59 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.goodchoice.kr:443
2021-03-11 18:08:59 [urllib3.connectionpool] DEBUG: https://www.goodchoice.kr:443 "POST /product/get_review_non HTTP/1.1" 200 None
2021-03-11 18:08:59 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=6525&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '3성급',
 'link': 'https://www.goodchoice.kr/product/detail?ano=6525&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12',
 'location': '서울 서초구 서초동 1361-10',
 'name': '호텔 페이토 강남',
 'platform': '여기어때',
 'price': '77,000원',
 'review': '239',
 'roomtype': '디럭스 트윈',
 'score': '9.0'}
2021-03-11 18:08:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.goodchoice.kr/product/detail?ano=10693&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12> (referer: https://www.goodchoice.kr/product/search/2/2019?sort=HIT%20%20%20%20%20%20%20%20%20%20%20%20&sel_date=2021-03-11&sel_date2=2021-03

2021-03-11 18:09:00 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.goodchoice.kr:443
2021-03-11 18:09:00 [urllib3.connectionpool] DEBUG: https://www.goodchoice.kr:443 "POST /product/get_review_non HTTP/1.1" 200 None
2021-03-11 18:09:00 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=9956&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '비지니스',
 'link': 'https://www.goodchoice.kr/product/detail?ano=9956&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12',
 'location': '서울 서초구 서초동 1593-1',
 'name': '서초 E호텔',
 'platform': '여기어때',
 'price': '60,000원',
 'review': '65',
 'roomtype': '[18시체크인] 스탠다드 더블 (본관)',
 'score': '8.9'}
2021-03-11 18:09:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.goodchoice.kr/product/detail?ano=48212&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12> (referer: https://www.goodchoice.kr/product/search/2/2018?sort=HIT%20%20%20%20%20%20%20%20%20%20%20%20&sel_date=2021-03-11&sel_da

2021-03-11 18:09:01 [urllib3.connectionpool] DEBUG: https://www.goodchoice.kr:443 "POST /product/get_review_non HTTP/1.1" 200 None
2021-03-11 18:09:01 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=48212&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '1성급',
 'link': 'https://www.goodchoice.kr/product/detail?ano=48212&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12',
 'location': '서울 은평구 응암동 30-2',
 'name': '은평 씨에스 에비뉴 호텔',
 'platform': '여기어때',
 'price': '50,000원',
 'review': '198',
 'roomtype': '디럭스 트윈',
 'score': '9.0'}
2021-03-11 18:09:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.goodchoice.kr/product/detail?ano=55968&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12> (referer: https://www.goodchoice.kr/product/search/2/2019?sort=HIT%20%20%20%20%20%20%20%20%20%20%20%20&sel_date=2021-03-11&sel_date2=2021-03-12&persons=2)
2021-03-11 18:09:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.goodchoice.k

2021-03-11 18:09:02 [urllib3.connectionpool] DEBUG: https://www.goodchoice.kr:443 "POST /product/get_review_non HTTP/1.1" 200 None
2021-03-11 18:09:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=58220&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '2성급',
 'link': 'https://www.goodchoice.kr/product/detail?ano=58220&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12',
 'location': '서울 은평구 갈현동 393-9',
 'name': '호텔 디 아티스트 연신내점',
 'platform': '여기어때',
 'price': '55,000원',
 'review': '30',
 'roomtype': '스탠다드 (18시 체크인)',
 'score': '9.2'}
2021-03-11 18:09:02 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.goodchoice.kr:443
2021-03-11 18:09:03 [urllib3.connectionpool] DEBUG: https://www.goodchoice.kr:443 "POST /product/get_review_non HTTP/1.1" 200 None
2021-03-11 18:09:03 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=66825&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-1

...

2021-03-11 18:10:53 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.goodchoice.kr:443
2021-03-11 18:10:53 [urllib3.connectionpool] DEBUG: https://www.goodchoice.kr:443 "POST /product/get_review_non HTTP/1.1" 200 None
2021-03-11 18:10:53 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=6271&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '3성급',
 'link': 'https://www.goodchoice.kr/product/detail?ano=6271&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12',
 'location': '서울 송파구 잠실동 180',
 'name': '파로스 호텔',
 'platform': '여기어때',
 'price': '40,000원',
 'review': '408',
 'roomtype': '[반나절 호캉스-숙박불가] 스탠다드 더블 (객...',
 'score': '9.4'}
2021-03-11 18:10:53 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=6271&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '3성급',
 'link': 'https://www.goodchoice.kr/product/detail?ano=6271&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12'

2021-03-11 18:10:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.goodchoice.kr/product/detail?ano=47138&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12> (referer: https://www.goodchoice.kr/product/search/2/2019?sort=HIT%20%20%20%20%20%20%20%20%20%20%20%20&sel_date=2021-03-11&sel_date2=2021-03-12&persons=2)
2021-03-11 18:10:55 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.goodchoice.kr:443
2021-03-11 18:10:55 [urllib3.connectionpool] DEBUG: https://www.goodchoice.kr:443 "POST /product/get_review_non HTTP/1.1" 200 None
2021-03-11 18:10:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=47138&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '부티크',
 'link': 'https://www.goodchoice.kr/product/detail?ano=47138&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12',
 'location': '서울 서초구 서초동 1594-6',
 'name': '나인스 호텔',
 'platform': '여기어때',
 'price': '65,000원',
 'review': '73',
 'roomtype': '골드 더블',
 'scor

2021-03-11 18:10:57 [urllib3.connectionpool] DEBUG: https://www.goodchoice.kr:443 "POST /product/get_review_non HTTP/1.1" 200 None
2021-03-11 18:10:57 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=64700&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '부티크',
 'link': 'https://www.goodchoice.kr/product/detail?ano=64700&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12',
 'location': '서울 서초구 서초동 1554-11',
 'name': '서초 두꺼비호텔',
 'platform': '여기어때',
 'price': '70,588원',
 'review': '0',
 'roomtype': '스탠다드',
 'score': ['점수없음']}
2021-03-11 18:10:57 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.goodchoice.kr:443
2021-03-11 18:10:57 [urllib3.connectionpool] DEBUG: https://www.goodchoice.kr:443 "POST /product/get_review_non HTTP/1.1" 200 None
2021-03-11 18:10:57 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=10670&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level':

2021-03-11 18:10:58 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=59067&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '5성급',
 'link': 'https://www.goodchoice.kr/product/detail?ano=59067&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12',
 'location': '서울 송파구 신천동 29',
 'name': '시그니엘 서울',
 'platform': '여기어때',
 'price': '459,800원',
 'review': '109',
 'roomtype': '그랜드 디럭스 (시티뷰)',
 'score': '9.9'}
2021-03-11 18:10:58 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=6716&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '레지던스',
 'link': 'https://www.goodchoice.kr/product/detail?ano=6716&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12',
 'location': '서울 서초구 서초동 1337-3',
 'name': '[반짝특가] 어반 플레이스 강남',
 'platform': '여기어때',
 'price': '55,000원',
 'review': '540',
 'roomtype': '[파격특가] 스탠다드 더블',
 'score': '8.8'}
2021-03-11 18:10:58 [scrapy.core.scraper] DEBUG: Scraped from <200 https://ww

2021-03-11 18:10:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.goodchoice.kr/product/detail?ano=6280&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12> (referer: https://www.goodchoice.kr/product/search/2/2019?sort=HIT%20%20%20%20%20%20%20%20%20%20%20%20&sel_date=2021-03-11&sel_date2=2021-03-12&persons=2)
2021-03-11 18:10:59 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.goodchoice.kr:443
2021-03-11 18:10:59 [urllib3.connectionpool] DEBUG: https://www.goodchoice.kr:443 "POST /product/get_review_non HTTP/1.1" 200 None
2021-03-11 18:10:59 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.goodchoice.kr/product/detail?ano=6280&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12>
{'level': '4성급',
 'link': 'https://www.goodchoice.kr/product/detail?ano=6280&adcno=2&sel_date=2021-03-11&sel_date2=2021-03-12',
 'location': '서울 서초구 양재동 202',
 'name': '더케이 호텔 서울',
 'platform': '여기어때',
 'price': '77,900원',
 'review': '509',
 'roomtype': '디럭스 트윈',
 'score

In [11]:
# 실행시간 2분 5초

In [12]:
data = pd.read_csv('goodchoice/goodchoice.csv', encoding='utf-8-sig')
print(data.shape)
data.tail()

(2390, 9)


,level,link,location,name,platform,price,review,roomtype,score
2385,3성급,https://www.goodchoice.kr/product/detail?ano=4...,서울 서초구 서초동 1339-1,신라스테이 서초,여기어때,"142,560원",581,[조식 2인 패키지] 디럭스 더블,9.1
2386,4성급,https://www.goodchoice.kr/product/detail?ano=6...,서울 서초구 양재동 202,더케이 호텔 서울,여기어때,"77,900원",509,디럭스 트윈,8.7
2387,4성급,https://www.goodchoice.kr/product/detail?ano=6...,서울 서초구 양재동 202,더케이 호텔 서울,여기어때,"88,000원",509,비즈니스 더블,8.7
2388,4성급,https://www.goodchoice.kr/product/detail?ano=6...,서울 서초구 양재동 202,더케이 호텔 서울,여기어때,"88,000원",509,[Coffee 패키지] 디럭스 트윈,8.7
2389,4성급,https://www.goodchoice.kr/product/detail?ano=6...,서울 서초구 양재동 202,더케이 호텔 서울,여기어때,"95,000원",509,[와인 패키지] 디럭스 트윈,8.7
